In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath('')))

import requests
import logging
import json
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import selenium
from lxml import html
from collections import defaultdict

from utils.constants import *

## stateofthedapps

In [69]:
base_url = "https://www.stateofthedapps.com/rankings/platform/"
query = "?page="

blockchain_query_dict = {
    "ethereum": 41,
    "bsc": 4
}

In [76]:
dapp_dict = defaultdict(list)

for platform, max_page in blockchain_query_dict.items():
    page_start = 1
    
    for page_number in tqdm(range(page_start, max_page+1), position=0, leave=True):
        url = f"{base_url}{platform}{query}{page_number}"
        req = requests.get(url).text

        soup = bs(req, 'html.parser')
        table_rows = soup.select(".table-data")

        for row in table_rows:
            dapp_name = row.select_one(".name").text
            dapp_dict[platform].append(dapp_name)

100%|█████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.36it/s]


In [161]:
with open('./data/stateofthedapp/dapp_list.txt', 'w') as f:
    json.dump(dapp_dict, f)

## vrdb

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options


In [3]:
url = "https://vrdb.app/quest/lab/index_us.html"

In [8]:
s=Service(ChromeDriverManager().install())

options = Options()
options.headless = True
driver = webdriver.Chrome(service=s, options=options)
driver.maximize_window()
driver.get(url)
vr_games = defaultdict(dict)
w = WebDriverWait(driver,3);
while True:
    tbody = w.until(EC.presence_of_element_located((By.TAG_NAME, 'tbody')));
    time.sleep(5)
    trs = tbody.find_elements(By.TAG_NAME, 'tr')
    for tr in trs:
        tds = tr.find_elements(By.TAG_NAME, 'td')
        
        
        game_title = tds[1].text
        vr_games[game_title]['title'] = game_title
        vr_games[game_title]['link'] = tds[1].find_element(By.TAG_NAME, 'a').get_attribute('href')
        vr_games[game_title]['price'] = tds[5].text
        vr_games[game_title]['release_date'] = tds[8].text
        vr_games[game_title]['genre'] = tds[9].text
        vr_games[game_title]['size'] = tds[10].text

    last_number = w.until(EC.presence_of_element_located((By.XPATH, "//a[contains(@class, 'paginate_button')][contains(text(), '23')]")))
    if "current" in last_number.get_attribute("class"):
        break
    driver.find_element(By.XPATH, "//a[contains(@id, 'oculuslist_next')]").click()
print("done")
driver.close()



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/jackyangara/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


done


In [10]:
with open('../data/vrdb/100622/vr.json', 'w') as f:
    json.dump(vr_games, f)


### get description and other info

In [ ]:
with open('../data/vrdb/100622/vr.json', 'r') as f:
    vr_games = json.load(f)


In [12]:
import urllib.request


In [21]:
failed

[]

In [25]:
s=Service(ChromeDriverManager().install())

options = Options()
options.headless = True

driver = webdriver.Chrome(service=s, options=options)
driver.maximize_window()
vr_complete_data = defaultdict(dict)
failed = []

for k, v in vr_games.items():
    # try:
        driver.get(v['link'])
        w = WebDriverWait(driver,5);
        close_modal = "//i[contains(@class, 'sky-modal__title-close-icon')]"
        w.until(EC.presence_of_element_located((By.XPATH, close_modal))).click()
        time.sleep(2)
        description_classname = "clamped-description__content"
        desc = w.until(EC.presence_of_element_located((By.CLASS_NAME, description_classname)))
        
        image_classname = "carousel--collapsed"
        image = w.until(EC.presence_of_element_located((By.CLASS_NAME, image_classname))).get_attribute('style')
        url = image.lstrip('background-image: url("').rstrip('");');
        urllib.request.urlretrieve(url, f"../data/vrdb/images/{k.lower().replace(' ', '_').replace('/','_')}.jpg")
            
        vr_complete_data[v['title']]['image'] =  f"../data/vrdb/images/{k.lower().replace(' ', '_').replace('/','_')}.jpg"
        
        vr_complete_data[v['title']]['title'] = v['title']
        vr_complete_data[v['title']]['desc'] = desc.text

        app_details_classname = "app-details__rows"

        app_detail_key_classname = "app-details-row__left"
        app_detail_value_classname = "app-details-row__right"

        app_details = w.until(EC.presence_of_element_located((By.CLASS_NAME, app_details_classname)))

        app_detail_key = [i.text for i in app_details.find_elements(By.CLASS_NAME, app_detail_key_classname)]
        app_detail_value = [i.text for i in app_details.find_elements(By.CLASS_NAME, app_detail_value_classname)]
        
        
        
        vr_complete_data[v['title']]['additional_data'] = dict(zip(app_detail_key, app_detail_value))
    # except Exception as e:
    #     failed.append(f"{k} {e}")
    # break
driver.close()



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/jackyangara/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


WebDriverException: Message: disconnected: Unable to receive message from renderer
  (Session info: headless chrome=102.0.5005.115)
Stacktrace:
0   chromedriver                        0x000000010c3ecd19 chromedriver + 5197081
1   chromedriver                        0x000000010c37ab23 chromedriver + 4729635
2   chromedriver                        0x000000010bf582b8 chromedriver + 393912
3   chromedriver                        0x000000010bf42e02 chromedriver + 306690
4   chromedriver                        0x000000010bf41d5a chromedriver + 302426
5   chromedriver                        0x000000010bf4225c chromedriver + 303708
6   chromedriver                        0x000000010bf6222b chromedriver + 434731
7   chromedriver                        0x000000010bf5abb8 chromedriver + 404408
8   chromedriver                        0x000000010bf5a8be chromedriver + 403646
9   chromedriver                        0x000000010bf5aff2 chromedriver + 405490
10  chromedriver                        0x00

In [23]:
vr_complete_data

defaultdict(dict,
            {'Hamster Run': {'image': 'hamster_run.jpg',
              'title': 'Hamster Run',
              'desc': 'this is hamster run, u can walk, jump, and run like a hamster, with over like around 5 minigames and 8 cosmetics u can choose from.',
              'additional_data': {'Game Modes': 'Multiplayer',
               'Supported Player Modes': 'Roomscale',
               'Supported Controllers': 'Oculus Touch',
               'Supported Platforms': 'Quest, Quest 2',
               'Genres': 'Casual',
               'Languages': 'English',
               'Version': '1.0.38',
               'Developer': 'vr_and_pc',
               'Publisher': 'PartyHatStudios',
               'Website': 'https://sidequestvr.com/app/8128/hamster-run',
               'Release Date': 'June 8, 2022',
               'Developer Privacy Policy': 'Privacy Policy',
               'Space Required': '236 MB'}},
             'Hands of Globzop': {'image': 'hands_of_globzop.jpg',
         

In [12]:
vr_complete_data

defaultdict(dict, {})

In [10]:
vr_complete_data

defaultdict(dict,
            {'Hauntify Mixed Reality': {'image': 'background-image: url("https://scontent.oculuscdn.com/v/t64.5771-25/38974865_1316248788835172_2609510633059723749_n.jpg?stp=dst-jpg_q92_s2048x2048&_nc_cat=109&ccb=1-5&_nc_sid=79b88e&_nc_ohc=KdtsIuxznWwAX-HfwH6&_nc_ht=scontent.oculuscdn.com&oh=00_AT972ule572TSZuxhXPJf4Td5R_A045UKrHsZVSyepd9Mw&oe=6249831C");',
              'title': 'Hauntify Mixed Reality',
              'desc': "Hauntify MR can be played in any type of building including single or multistory!  The player must setup their play space using the game's built in features to enhance their experience. We want to give the player creative freedom to create a horror game and place props around their environment.  Some ghosts can be summoned in player defined spawn points and other ghosts appear in completely random locations around your house.  As you collect more relics, increasing dangerous spirits will be summoned.\n\nHauntify MR features fully dynamic AI tha

In [15]:
with open('../data/vrdb/vr_complete2.json', 'w') as f:
    json.dump(vr_complete_data, f)
    

# with open('../data/vrdb/failed.txt', 'w') as f:
#     failed =map(lambda x:x+'\n', failed)
#     f.writelines(failed)
    

## dapp radar

In [5]:
from utils.get_proxy import Proxy_List
from selenium.webdriver.common.proxy import Proxy, ProxyType

In [4]:
proxies = Proxy_List()

In [ ]:
s = Service(GeckoDriverManager().install())
fail_proxy_list = []
working_proxy_list = []
proxies = ["65.21.153.73:3007",
"138.68.60.8:8080",
"20.94.230.158:80",
"47.74.152.29:8888",
"107.151.182.247:80",
"20.94.229.106:80",
"108.62.49.26:3128",
"64.44.164.254:80",
"23.238.33.186:80",
"65.21.3.120:80",
"129.226.148.192:443",
"138.68.235.51:80",
"8.210.83.33:80",
"8.214.7.15:80",
"216.137.184.253:80",
"191.96.42.80:8080",
"185.101.97.24:8118",
"209.97.150.167:8080",
"165.227.71.60:80",
"8.217.2.51:59394",
"161.35.4.201:80",
"47.242.200.148:80",
"159.65.69.186:9300",
"3.222.173.245:80",
"173.82.149.243:8080",
"52.250.1.171:80",
"52.250.1.188:80",
"47.245.11.194:80",
"198.11.183.14:80"]
from selenium.webdriver.firefox.options import Options
for i in proxies:
    cap = set_proxy(i)
    try:
        
        firefox_options = Options()
        firefox_options.add_argument('--proxy-server=%s' % i)
        driver = webdriver.Firefox(service=s,options=firefox_options)
        
        driver.set_page_load_timeout(15)
        driver.get(f"https://dappradar.com/rankings/protocol/ethereum/{1}")
        w = WebDriverWait(driver,5);
        table = w.until(EC.presence_of_element_located((By.CLASS_NAME, 'rankings-table')));
    except Exception as e:
        print(e)
        fail_proxy_list.append(i)
    else:
        working_proxy_list.append(i)
    finally:
        driver.close()
    

In [63]:
import multiprocessing as mp

total_page = [i for i in range(1,126)]
total_page_2d = np.reshape(total_page, (-1, 25))
print(total_page_2d)
proxy_list = ["20.105.253.176:8080",
        "161.97.183.154:3128",
        "8.210.83.33:80",
        "8.210.221.30:80",
        "20.84.90.24:8214",
        "199.188.192.176:8080",
        "107.151.182.247:80",
        "18.216.136.190:9090"]
print(list(zip(total_page_2d, proxy_list)))
# output = mp.Queue()
# processes = [mp.Process(target=crawl_dapp_radar, args=(list(page_list))) for page_list in total_page_2d]

# for p in processes:
#     p.start()

# for p in processes:
#     p.join()

# print("process completed")


[[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
   19  20  21  22  23  24  25]
 [ 26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43
   44  45  46  47  48  49  50]
 [ 51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68
   69  70  71  72  73  74  75]
 [ 76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93
   94  95  96  97  98  99 100]
 [101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118
  119 120 121 122 123 124 125]]
[(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25]), '20.105.253.176:8080'), (array([26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42,
       43, 44, 45, 46, 47, 48, 49, 50]), '161.97.183.154:3128'), (array([51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75]), '8.210.83.33:80'), (array([ 76,  77,  78,  79,  80,  81,  82,  83,  84,  8

In [29]:
proxies = Proxy_List()


In [8]:
def set_proxy(p):

    pxy = Proxy()
    print(p)
    #set proxy type
    pxy.proxy_type = ProxyType.MANUAL

    #http proxy
    pxy.http_proxy = p

    #ssl proxy
    pxy.ssl_proxy = p

    #object of DesiredCapabilities
    c = webdriver.DesiredCapabilities.FIREFOX

    #set proxy browser capabilties
    pxy.add_to_capabilities(c)
    return c


In [56]:
s = Service(GeckoDriverManager().install())

for i in range(10):
    cap = set_proxy(i)
    driver = webdriver.Firefox(service=s, desired_capabilities=cap)
    driver.set_page_load_timeout(5)
    try:
        driver.get('http://example.com/')
        driver.find_element(By.TAG_NAME, 'div')
        print(i)
    except Exception as e:
        no_work.append(i)
    finally:
        driver.close()



====== WebDriver manager ======
Current firefox version is 97.0
Get LATEST geckodriver version for 97.0 firefox
Driver [/Users/jackyangara/.wdm/drivers/geckodriver/macos/v0.30.0/geckodriver] found in cache
/var/folders/zl/fh70qspj56q6ktktvql5f7gw0000gn/T/ipykernel_19257/3499147721.py:5: DeprecationWarning: capabilities and desired_capabilities have been deprecated, please pass in a Service object
  driver = webdriver.Firefox(service=s, desired_capabilities=cap)


8.217.112.99:59394
20.105.253.176:8080
1
161.97.183.154:3128
2
47.243.68.117:8080
8.210.83.33:80
4
8.210.221.30:80
5
20.84.90.24:8214
6
199.188.192.176:8080
7
107.151.182.247:80
8
18.216.136.190:9090
9


In [39]:
[1,2,4,5,6,7,8,9]

[]

In [ ]:
# schema 

{
    "OpenSea": 
        {
            "name": "OpenSea",
            "chain": "ETH",
            "category": "Marketplace",
            "balance": "some dollars",
            "users": "some users",
            "volume": "some volume",
            "social": ["social links"],
            "description": "some description",
            "smart contracts": [("smart contracts", "chain")],
        }
}
        

In [78]:
links_from_brand = brand.find_elements(By.TAG_NAME, 'li')

In [81]:
print(links_from_brand[0].find_element(By.TAG_NAME, 'a').get_attribute('href'))

https://twitter.com/projectopensea


In [59]:
!pip uninstall proxybroker -y

Found existing installation: proxybroker 0.3.2
Uninstalling proxybroker-0.3.2:
  Successfully uninstalled proxybroker-0.3.2


In [11]:
from selenium.webdriver.firefox.options import Options

In [37]:
from datetime import datetime
s = Service(GeckoDriverManager().install())

options = Options()
options.headless = True
dapp_dict = defaultdict(dict)


chain_classname = "//div[contains(@class, 'sc-ljMRFG') and contains(@class, 'ikDqTv')]"
dapp_classname = "//span[contains(@class, 'sc-hKumaY') and contains(@class, 'falIJP')]"
category_classname = "//div[contains(@class, 'rankings-column__category') and contains(@class, 'sc-eSxRXt') and contains(@class, 'fxjZId')]"
balance_classname = "//div[contains(@class, 'rankings-column__balance') and contains(@class, 'sc-eSxRXt') and contains(@class, 'fxjZId')]"
users_classname = "//div[contains(@class, 'rankings-column__users') and contains(@class, 'sc-eSxRXt') and contains(@class, 'fxjZId')]"
volume_classname = "//div[contains(@class, 'rankings-column__volume') and contains(@class, 'sc-eSxRXt') and contains(@class, 'fxjZId')]"
ad_classname =  "//div[contains(@class, 'sc-eZhRLC') and contains(@class, 'ihIlDT')]"
image_classname = "//div[contains(@class, 'sc-jNHqnW') and contains (@class, 'eZJvve')]"

page_list = [i for i in range(116,126)]
driver = webdriver.Firefox(service=s, desired_capabilities=None, options=options)
for page in page_list:
    start = datetime.now()
    driver.get(f"https://dappradar.com/rankings/protocol/ethereum/{page}")
    
    w = WebDriverWait(driver,10);

    for i in range((page-1)*25+1,(page-1)*25+26):
        
        table = w.until(EC.presence_of_element_located((By.XPATH, dapp_classname)));
        ad = table.find_elements(By.XPATH, ad_classname)
        names = table.find_elements(By.XPATH, dapp_classname)
        
        chains = table.find_elements(By.XPATH, chain_classname)
        category = table.find_elements(By.XPATH, category_classname)
        balance = table.find_elements(By.XPATH, balance_classname)
        users = table.find_elements(By.XPATH, users_classname)
        volume = table.find_elements(By.XPATH, volume_classname)
        image = table.find_elements(By.XPATH, image_classname)
        dapp_id_list = [a.text for a in ad]
        idx = dapp_id_list.index(str(i))
        dapp = names[idx].text
        print(dapp)
        dapp_dict[dapp]["dapp_id"] = idx
        dapp_dict[dapp]["name"] = names[idx].text
        dapp_dict[dapp]["chain"] = chains[idx].text
        dapp_dict[dapp]["category"] = category[idx].text
        dapp_dict[dapp]["balance"] = balance[idx].text
        dapp_dict[dapp]["users"] = users[idx].text
        dapp_dict[dapp]["volume"] = volume[idx].text
        dapp_dict[dapp]["imageUrl"] = image[idx].find_element(By.TAG_NAME, 'img').get_attribute('src')
        button = names[idx].find_element(By.XPATH, './../../../../..')
        link = button.get_attribute('href')
        driver.get(f"{link}")
        
        # social links
        brand_classname = "//div[contains(@class, 'article-page__brand-info')]"
        
        brand = w.until(EC.presence_of_element_located((By.XPATH, brand_classname)));
        social_parents = brand.find_elements(By.TAG_NAME, "li")
        social_links = [social_parent.find_element(By.TAG_NAME, 'a').get_attribute('href') for social_parent in social_parents]
        dapp_dict[dapp]['social'] = social_links
        
        # description -> get href
        desc_classname = "article-page__description"
        try:
            desc = w.until(EC.presence_of_element_located((By.CLASS_NAME, desc_classname))).text;
        except:
            desc = ""
        dapp_dict[dapp]['description'] = desc
        try:
        # open smart contract list
            sc_open_classname = "//div[contains(@class, 'sc-jNhWkn') and contains(@class, 'ccpdCd')]"
            sc_open = w.until(EC.presence_of_element_located((By.XPATH, sc_open_classname)));
            driver.execute_script("arguments[0].click();", sc_open)
        
        # get modal
            modal_classname = "modal-content"
            modal = w.until(EC.presence_of_element_located((By.CLASS_NAME, modal_classname)));
            # smart contract address
            sc_address_classname = "//a[contains(@class, 'sc-chSlKD') and contains(@class, 'sc-halPKt') and contains(@class, 'igpqrv') and contains(@class, 'ewcRJL')]"
        
            addresses = w.until(EC.presence_of_all_elements_located((By.XPATH, sc_address_classname)))
            addresses_unpacked = [address.text for address in addresses]
            sc_chain_classname = "//div[contains(@class, 'sc-ljMRFG') and contains(@class, 'ikDqTv')]"
            chains =  w.until(EC.presence_of_all_elements_located((By.XPATH, sc_chain_classname)))
            chains_unpacked = [chain.text for chain in chains]
            dapp_dict[dapp]['smart_contract'] = list(zip(addresses_unpacked, chains_unpacked))
        
            driver.find_element(By.CLASS_NAME, 'modal-close').click()
        except:
            dapp_dict[dapp]['smart_contract'] = []
            driver.find_element(By.CLASS_NAME, 'modal-close').click()
            
        open_dapp_classname = "//a[contains(@class, 'sc-kOyqGX') and contains(@class, 'hDjVuk')]"
        open_dapp = w.until(EC.presence_of_element_located((By.XPATH, open_dapp_classname)));
        driver.execute_script("arguments[0].click();", open_dapp)
        w.until(EC.number_of_windows_to_be(2))
        
        original_window = driver.current_window_handle
        for window_handle in driver.window_handles:
            if window_handle != original_window:
                driver.switch_to.window(window_handle)
                break
        try:
            WebDriverWait(driver, 5).until(lambda driver: driver.execute_script('return document.readyState') == 'complete')
        except:
            pass
        
        dapp_dict[dapp]['url'] = driver.current_url
        driver.close()
        
        driver.switch_to.window(original_window)
        
        driver.execute_script("window.history.go(-1)")
    with open(f"../data/dapp_radar/{page}.json", 'w') as f:
            json.dump(dapp_dict, f)
    print(f"took {datetime.now() - start} to complete")
    



====== WebDriver manager ======
Current firefox version is 97.0
Get LATEST geckodriver version for 97.0 firefox
Driver [/Users/jackyangara/.wdm/drivers/geckodriver/macos/v0.30.0/geckodriver] found in cache


The Front Pub
The HODL Community
The Illuminati
The Jigsaw Games
The Mask Society NFT
The Mike Tyson NFT Collection
The Million DeFi Homepage
The Million Ether Homepage
the Oracle
The Pyramid Game
The RAPSCALLIONS
The Richest Says
The SHILLcoin HODLcoin
The Tap Platform
The Train
TheButton
TheLast
TheNextBlock
Thetanuts
ThisOrThat.io
Thomas Crown Art "smART"
THOR
Thor Swap
Thousand Ether Goatse
TIBRIX
took 0:05:33.997065 to complete
Ticketh
Tides of Magic
Tigers Line
tigersline
Tinybit
Tip Top Universe
TKN.COM
To Be A King
To The Moon
Tok'n'talk
TOKEN BUYERS CLUB
Token ERC20 Generate
Token Fighter
Token Locker
Token Tycoon : Offical Release
token.store ETH
TokenField
TokenFlip
TokenMaker
TokenMaker
TokenPark
Tokensender
NEWTokensfarm
TokenSwap
Tokenvote
took 0:05:36.782198 to complete
TokenWarriors
Tom Sachs: ROCKET FACTORY PATCHES
TomorrowsPrice
TopBidder
Tornado Cash
Torpedo LAUNCH
Torum
TossCoin
TotalMoney
Totle
Totle
Totle Swap
Tracer DAO
TradePass
TradeStars
Tranche Finance
TREASU

TimeoutException: Message: 
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:183:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.jsm:395:5
element.find/</<@chrome://remote/content/marionette/element.js:300:16


<selenium.webdriver.remote.webelement.WebElement (session="8ee8d98b-b449-4906-bb7d-935a380ffcbe", element="85b36f09-2a46-4268-9a60-5340af1d5bfa")>

In [21]:

    with open(f"../data/dapp_radar/15.json", 'w') as f:
            json.dump(dapp_dict, f)

In [ ]:
dapp_dict['Axie Infinity']

In [13]:
from datetime import datetime
start = datetime.now()
print(f"took {datetime.now() - start} to complete")

took 0:00:00.000072 to complete


In [94]:
with open('./data/dapp_radar/page_1.json', 'w') as f:
    json.dump(dapp_dict, f)

In [ ]:
dapp_dict

In [28]:
with open('../data/dapp_radar/new_dapp.json', 'w') as f:
    json.dump(dapp_dict, f)

In [5]:
def crawl_dapp_radar(page_list, proxy):
    
    s = Service(GeckoDriverManager().install())
    cap = set_proxy(proxy)

    dapp_dict = defaultdict(dict)

    chain_classname = "//div[contains(@class, 'sc-dMOJrz') and contains(@class, 'gcSIev')]"
    dapp_classname = "//span[contains(@class, 'sc-fSDTwv') and contains(@class, 'hRDPRw')]"
    category_classname = "//div[contains(@class, 'rankings-column__category') and contains(@class, 'sc-jNHqnW') and contains(@class, 'bIoQbB')]"
    balance_classname = "//div[contains(@class, 'rankings-column__balance') and contains(@class, 'sc-jNHqnW') and contains(@class, 'bIoQbB')]"
    users_classname = "//div[contains(@class, 'rankings-column__users') and contains(@class, 'sc-jNHqnW') and contains(@class, 'bIoQbB')]"
    volume_classname = "//div[contains(@class, 'rankings-column__volume') and contains(@class, 'sc-jNHqnW') and contains(@class, 'bIoQbB')]"
    ad_classname =  "//div[contains(@class, 'sc-iRFsWr') and contains(@class, 'fpCOlK')]"
    image_classname = "//div[contains(@class, 'sc-cqJhZP') and contains (@class, 'jxncUU')]"

    driver = webdriver.Firefox(service=s, desired_capabilities=cap)
    for page in page_list:
        
        logging.info(f"page: {page}")
        driver.get(f"https://dappradar.com/rankings/protocol/ethereum/{page}")

        w = WebDriverWait(driver,10);

        for i in range((page-1)*25+1,(page-1)*25+26):
            try: 
                table = w.until(EC.presence_of_element_located((By.XPATH, dapp_classname)));
                ad = table.find_elements(By.XPATH, ad_classname)
                names = table.find_elements(By.XPATH, dapp_classname)

                chains = table.find_elements(By.XPATH, chain_classname)
                category = table.find_elements(By.XPATH, category_classname)
                balance = table.find_elements(By.XPATH, balance_classname)
                users = table.find_elements(By.XPATH, users_classname)
                volume = table.find_elements(By.XPATH, volume_classname)
                image = table.find_elements(By.XPATH, image_classname)
                dapp_id_list = [a.text for a in ad]
                idx = dapp_id_list.index(str(i))
                dapp = names[idx].text
                dapp_dict[dapp]["dapp_id"] = idx
                dapp_dict[dapp]["name"] = names[idx].text
                dapp_dict[dapp]["chain"] = chains[idx].text
                dapp_dict[dapp]["category"] = category[idx].text
                dapp_dict[dapp]["balance"] = balance[idx].text
                dapp_dict[dapp]["users"] = users[idx].text
                dapp_dict[dapp]["volume"] = volume[idx].text
                dapp_dict[dapp]["imageUrl"] = image[idx].find_element(By.TAG_NAME, 'img').get_attribute('src')
                button = names[idx].find_element(By.XPATH, './../../../../..')
                link = button.get_attribute('href')
                driver.get(f"{link}")

                # social links
                brand_classname = "//div[contains(@class, 'article-page__brand-info')]"

                brand = w.until(EC.presence_of_element_located((By.XPATH, brand_classname)));
                social_parents = brand.find_elements(By.TAG_NAME, "li")
                social_links = [social_parent.find_element(By.TAG_NAME, 'a').get_attribute('href') for social_parent in social_parents]
                dapp_dict[dapp]['social'] = social_links

                # description -> get href
                desc_classname = "article-page__description"
                desc = w.until(EC.presence_of_element_located((By.CLASS_NAME, desc_classname))).text;
                dapp_dict[dapp]['description'] = desc

                # open smart contract list
                sc_open_classname = "//div[contains(@class, 'sc-jNhWkn') and contains(@class, 'ccpdCd')]"
                sc_open = w.until(EC.presence_of_element_located((By.XPATH, sc_open_classname)));
                driver.execute_script("arguments[0].click();", sc_open)

                # get modal
                modal_classname = "modal-content"
                modal = w.until(EC.presence_of_element_located((By.CLASS_NAME, modal_classname)));
                # smart contract address
                sc_address_classname = "//a[contains(@class, 'sc-chSlKD') and contains(@class, 'sc-halPKt') and contains(@class, 'igpqrv') and contains(@class, 'ewcRJL')]"

                addresses = w.until(EC.presence_of_all_elements_located((By.XPATH, sc_address_classname)))
                addresses_unpacked = [address.text for address in addresses]
                sc_chain_classname = "//div[contains(@class, 'sc-kdneuM') and contains(@class, 'jYHpMl')]"
                chains =  w.until(EC.presence_of_all_elements_located((By.XPATH, sc_chain_classname)))
                chains_unpacked = [chain.text for chain in chains]
                dapp_dict[dapp]['smart_contract'] = list(zip(addresses_unpacked, chains_unpacked))

                driver.find_element(By.CLASS_NAME, 'modal-close').click()

                open_dapp_classname = "//a[contains(@class, 'sc-iiCSmI') and contains(@class, 'kLgVkv')]"
                open_dapp = w.until(EC.presence_of_element_located((By.XPATH, open_dapp_classname)));
                driver.execute_script("arguments[0].click();", open_dapp)
                w.until(EC.number_of_windows_to_be(2))

                original_window = driver.current_window_handle
                for window_handle in driver.window_handles:
                    if window_handle != original_window:
                        driver.switch_to.window(window_handle)
                        break

                WebDriverWait(driver, 10).until(lambda driver: driver.execute_script('return document.readyState') == 'complete')

                dapp_dict[dapp]['url'] = driver.current_url
                driver.close()

                driver.switch_to.window(original_window)

                driver.execute_script("window.history.go(-1)")
                
                logging.info(f"done: {page}, {i}")
            except:
                logging.error(f"error: {page}, {i}")
            
        with open(f"./data/dapp_radar/{page}.json", 'w') as f:
            json.dump(dapp_dict, f)
import multiprocessing as mp


    
logging.basicConfig(file='./logs/dapp_radar/error.log')
logging.basicConfig(level=logging.INFO, format='%(asctime)s :: %(levelname)s :: %(message)s')

total_page = [i for i in range(1,126)]
total_page_2d = np.reshape(total_page, (-1, 25))
print(total_page_2d)
proxy_list = ["20.105.253.176:8080",
    "161.97.183.154:3128",
    "8.210.83.33:80",
    "8.210.221.30:80",
    "20.84.90.24:8214",
    "199.188.192.176:8080",
    "107.151.182.247:80",
    "18.216.136.190:9090"]
page_and_proxy = (list(zip(total_page_2d, proxy_list)))
output = mp.Queue()
processes = [mp.Process(target=crawl_dapp_radar, args=(page_list, proxy)) for page_list, proxy in page_and_proxy]

for p in processes:
    p.start()

for p in processes:
    p.join()

print("process completed")


[[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
   19  20  21  22  23  24  25]
 [ 26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43
   44  45  46  47  48  49  50]
 [ 51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68
   69  70  71  72  73  74  75]
 [ 76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93
   94  95  96  97  98  99 100]
 [101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118
  119 120 121 122 123 124 125]]
process completed


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'crawl_dapp_radar' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'crawl_dapp_radar' on <module '__main__' (built-in)>
Traceback (most 

## etherscan